In [ ]:
import os
from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient
from dlt.sources.filesystem import readers
import dlt

load_dotenv()

In [ ]:
pipeline = dlt.pipeline(
    pipeline_name="minio_to_azure",
    destination="filesystem",
    dataset_name="dimelo_flow",
)

In [3]:
parquet_reader = readers(
    bucket_url="s3://my-bucket/taxis/taxis_iceberg_5128d68f-aa5c-4270-a561-d530f60229b4/data",
    file_glob="*.parquet"
).read_parquet()

parquet_reader = parquet_reader.with_name("df_parquet")

In [4]:
# Ejecuta el pipeline
load_info = pipeline.run(
    parquet_reader,
    loader_file_format="parquet",
    write_disposition="replace"
)

print(load_info)
print(pipeline.last_trace.last_normalize_info)

Pipeline minio_to_azure load step completed in 3 minutes and 16.32 seconds
1 load package(s) were loaded to destination filesystem and into dataset dimelo_flow
The filesystem destination used abfss://clase-4-dlt@fhbd.dfs.core.windows.net/GRUPO_6 location to store data
Load package 1757139071.3071916 is LOADED and contains no failed jobs
Normalized data for the following tables:
- df_parquet: 3475226 row(s)
- _dlt_pipeline_state: 1 row(s)

Load package 1757139071.3071916 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs


## Confirmar subida del archivo en bucket de Azure

In [ ]:
# Configuración desde variables de entorno
ACCOUNT_NAME = os.getenv("AZURE_STORAGE_ACCOUNT_NAME")
ACCOUNT_KEY = os.getenv("AZURE_STORAGE_ACCOUNT_KEY")
CONTAINER_NAME = os.getenv("AZURE_CONTAINER_NAME")

def list_azure_blobs():
    """Lista todos los blobs del contenedor"""
    try:
        # Crear cliente
        blob_service_client = BlobServiceClient(
            account_url=f"https://{ACCOUNT_NAME}.blob.core.windows.net",
            credential=ACCOUNT_KEY
        )
        
        # Obtener contenedor
        container_client = blob_service_client.get_container_client(CONTAINER_NAME)
        
        # Listar blobs
        print(f"Objetos en el contenedor '{CONTAINER_NAME}':")
        print("-" * 50)
        
        for i, blob in enumerate(container_client.list_blobs(), 1):
            size_mb = blob.size / (1024 * 1024)
            print(f"{i:3d}. {blob.name} ({size_mb:.2f} MB)")
        
    except Exception as e:
        print(f"Error: {e}")

In [ ]:
list_azure_blobs()

Objetos en el contenedor 'clase-4-dlt':
--------------------------------------------------
  1. GRUPO_1 (0.00 MB)
  2. GRUPO_2 (0.00 MB)
  3. GRUPO_3 (0.00 MB)
  4. GRUPO_4 (0.00 MB)
  5. GRUPO_4/azure (0.00 MB)
  6. GRUPO_4/azure/_dlt_loads (0.00 MB)
  7. GRUPO_4/azure/_dlt_loads/s3_to_adls__1756955062.0879076.jsonl (0.00 MB)
  8. GRUPO_4/azure/_dlt_loads/s3_to_adls__1756955439.6662495.jsonl (0.00 MB)
  9. GRUPO_4/azure/_dlt_loads/s3_to_adls__1756955762.6651325.jsonl (0.00 MB)
 10. GRUPO_4/azure/_dlt_loads/s3_to_adls__1756957266.020199.jsonl (0.00 MB)
 11. GRUPO_4/azure/_dlt_loads/s3_to_adls__1757016731.154526.jsonl (0.00 MB)
 12. GRUPO_4/azure/_dlt_loads/s3_to_adls__1757017205.9115121.jsonl (0.00 MB)
 13. GRUPO_4/azure/_dlt_pipeline_state (0.00 MB)
 14. GRUPO_4/azure/_dlt_pipeline_state/s3_to_adls__1756955062.0879076__b87e3316b2d9e3007f16a0659141e6b397338cc68bfd756b06fd9864ab857b50.jsonl (0.00 MB)
 15. GRUPO_4/azure/_dlt_pipeline_state/s3_to_adls__1757017205.9115121__9e3546ac95885510